# Imports

In [295]:
import pandas as pd

In [296]:
import xml.etree.ElementTree as et

In [297]:
import os
import sys

In [298]:
import matplotlib.pyplot as plt

In [299]:
#import argparse

# Define and parse command-line arguments
#parser = argparse.ArgumentParser(description='Process some integers.')
#parser.add_argument('--seed', type=int, help='Seed value')
#parser.add_argument('--traffic_scale', type=float, help='Traffic scale value')
#args = parser.parse_args()

# Functions

In [300]:
def get_last_folder(path):
    # Normalize the path to handle different separators and remove trailing separator
    normalized_path = os.path.normpath(path)
    # Split the path into components
    folders = normalized_path.split(os.sep)
    # Get the last folder
    last_folder = folders[-1]
    return last_folder

In [301]:
price_per_wh = 70 / 1000

In [302]:
def get_price(d_energy):
    return d_energy * price_per_wh

In [303]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict


In [304]:
def calc_elevation_up(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    up = z_diff.apply(lambda x: x if x > 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_up = up.sum()
    return total_up

In [305]:
def calc_elevation_down(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    down = z_diff.apply(lambda x: x if x < 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_down = down.sum()
    return total_down

In [306]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict

In [307]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        route_dict['stops'] = stops
        
        yield route_dict

In [308]:
def get_group_by_id(list_of_dfs, desired_id):
    for df in list_of_dfs:
        if desired_id in df['id'].values:
            return df[df['id'] == desired_id]
    raise ValueError(f"ID '{desired_id}' not found in any dataframe.")

# XML to df

In [309]:
base_folder = "C:\\Users\\Admin\\Sumo\\nap_gellert_b"

In [310]:
file_path = os.path.join(base_folder, "emission.out.xml")
emission_output = et.parse(file_path)

transform = transform_xml(emission_output.getroot())
emission_output_list = list(transform)

emission_output_df = pd.DataFrame(emission_output_list)
emission_output_df = emission_output_df.drop(emission_output_df.columns[0], axis=1)

emission_output_df.shape

(40075, 20)

In [311]:
file_path = os.path.join(base_folder, "Battery.out.xml")

battery_output = et.parse(file_path)
battery_output_root = battery_output.getroot()

transform = transform_xml(battery_output_root)
battery_output_list = list(transform)

battery_output_pd = pd.DataFrame(battery_output_list)

battery_output_pd = battery_output_pd.drop(battery_output_pd.columns[0], axis=1)
battery_output_pd

,id,energyConsumed,totalEnergyConsumed,totalEnergyRegenerated,actualBatteryCapacity,maximumBatteryCapacity,chargingStationId,energyCharged,energyChargedInTransit,energyChargedStopped,speed,acceleration,x,y,lane,posOnLane,timeStopped
0,C76647148.0,0.0000,0.0000,0.0000,17500.0000,35000.0000,NULL,0.0000,0.0000,0.0000,0.0000,0.0000,4059.1679,1645.2121,-894159520#0_0,12.1000,0
1,C76647148.0,0.5743,0.5743,0.0000,17499.4257,35000.0000,NULL,0.0000,0.0000,0.0000,0.9320,0.9320,4058.3237,1644.8207,-894159520#0_0,13.0320,0
2,C76647148.0,1.5558,2.1301,0.0000,17497.8699,35000.0000,NULL,0.0000,0.0000,0.0000,2.0210,1.0890,4056.4934,1643.9718,-894159520#0_0,15.0531,0
3,C76647148.0,2.4520,4.5821,0.0000,17495.4179,35000.0000,NULL,0.0000,0.0000,0.0000,3.0631,1.0421,4053.7192,1642.6853,-894159520#0_0,18.1162,0
4,C76647148.0,2.3521,6.9342,0.0000,17493.0658,35000.0000,NULL,0.0000,0.0000,0.0000,3.6727,0.6096,4050.3929,1641.1428,-894159520#0_0,21.7889,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40070,C78105432.0,7.4367,672.8787,340.7216,17167.8428,35000.0000,NULL,0.0000,0.0000,0.0000,7.9794,0.5798,3705.2034,3226.7254,25440988#0_0,52.6439,0
40071,C78105432.0,6.4637,679.3425,340.7216,17161.3791,35000.0000,NULL,0.0000,0.0000,0.0000,8.2581,0.2787,3698.1904,3222.4658,25440988#0_0,60.9020,0
40072,C78105432.0,5.0547,684.3972,340.7216,17156.3244,35000.0000,NULL,0.0000,0.0000,0.0000,8.2110,-0.0471,3691.2175,3218.2306,25440988#0_0,69.1129,0
40073,C78105432.0,3.5808,687.9780,340.7216,17152.7436,35000.0000,NULL,0.0000,0.0000,0.0000,7.8633,-0.3477,3684.5399,3214.1746,25440988#0_0,76.9762,0


In [312]:
file_path = os.path.join(base_folder, "tripinfo.xml")

b_tripinfo_output = et.parse(file_path)
b_tripinfo_output_root = b_tripinfo_output.getroot()

transform = transform_xml_tripinfo(b_tripinfo_output_root)
b_tripinfo_output_list = list(transform)

b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)

b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
b_tripinfo_output_pd

,id,depart,departLane,departPos,departSpeed,departDelay,arrival,arrivalLane,arrivalPos,arrivalSpeed,...,routeLength,waitingTime,waitingCount,stopTime,timeLoss,rerouteNo,devices,vType,speedFactor,vaporized
0,C76647148.0,23690.00,-894159520#0_0,12.10,0.00,0.00,24087.00,84596611#2_0,420.96,0.62,...,1612.25,139.00,7,40.00,240.90,0,tripinfo_C76647148.0 emissions_C76647148.0 bat...,bus,1.00,
1,C6359838.0,24410.00,34572881#1_0,12.10,0.00,0.00,24597.00,34572878#1_0,48.32,5.85,...,673.45,56.00,4,20.00,109.62,0,tripinfo_C6359838.0 emissions_C6359838.0 batte...,bus,1.00,
2,C7644455.0,26490.00,894166783_0,4.14,0.00,40.00,26723.00,311385575_0,48.95,2.05,...,1077.56,52.00,5,30.00,113.39,0,tripinfo_C7644455.0 emissions_C7644455.0 batte...,bus,1.00,
3,C78144121.0,30470.00,34800660#4_0,12.10,0.00,0.00,30805.00,1084897573#5_0,49.55,5.28,...,1904.06,41.00,6,60.00,137.59,0,tripinfo_C78144121.0 emissions_C78144121.0 bat...,bus,1.00,
4,C76796162.0,31250.00,53864729#4_0,12.10,0.00,0.00,31314.00,53864729#4_1,445.46,0.72,...,433.36,1.00,1,20.00,12.73,0,tripinfo_C76796162.0 emissions_C76796162.0 bat...,bus,1.00,
5,C74509178.0,31310.00,262231193#4_0,12.10,0.00,0.00,33858.00,63682647_0,126.86,0.91,...,3973.37,1912.00,21,110.00,2138.63,0,tripinfo_C74509178.0 emissions_C74509178.0 bat...,bus,1.00,
6,C76796194.0,34550.00,37390256#0_0,12.10,0.00,0.00,34623.00,127553879_0,21.93,3.73,...,262.10,3.00,2,20.00,34.14,0,tripinfo_C76796194.0 emissions_C76796194.0 bat...,bus,1.00,
7,C78245617.0,37370.00,-465237877#2_0,12.10,0.00,0.00,40775.00,-581325727_0,332.48,8.77,...,8104.52,1786.00,17,617.00,2097.55,0,tripinfo_C78245617.0 emissions_C78245617.0 bat...,bus,1.00,
8,C78144138.0,40850.00,170104270_0,12.10,0.00,0.00,41516.00,34800660#4_0,47.72,6.61,...,2706.75,122.00,10,191.00,276.39,0,tripinfo_C78144138.0 emissions_C78144138.0 bat...,bus,1.00,
9,C76796318.0,42830.00,37390256#0_0,12.10,0.00,0.00,42893.00,127553879_0,21.93,3.75,...,262.10,0.00,0,21.00,23.09,0,tripinfo_C76796318.0 emissions_C76796318.0 bat...,bus,1.00,


In [313]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

stops_pd = pd.DataFrame(stops_list)

In [314]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
#vehicles_pd

## Grouping by id

In [315]:
grouped_df = battery_output_pd.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]
#C74509106 = list_of_dfs[0]
#C74509124 = list_of_dfs[1]
#C74509135 = list_of_dfs[2]

In [316]:
grouped_emission_df = emission_output_df.groupby('id')

list_of_emission_dfs = [group_data for _, group_data in grouped_emission_df]
#list_of_emission_dfs



In [322]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = get_price(float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1]))
    print(float(group_data['totalEnergyConsumed'].iloc[-1]))
    
    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    z_up = calc_elevation_up(get_group_by_id(list_of_emission_dfs, group_id))
    z_down = calc_elevation_down(get_group_by_id(list_of_emission_dfs, group_id))
    
    # Store the results in a dictionary
    group_result = {
        'id': group_id,
        'avgSpeed': avg_speed,
        'battery': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops,
        'up': z_up,
        'down': z_down
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
print(result_df)


229.6697
1097.0766
1041.5067
1431.525
1466.0374
1113.4287
835.2638
654.4894
846.4561
339.6552
472.1351
364.4948
110.9746
63.0955
70.9013
215.0349
3251.5744
863.9859
1098.9907
694.8298
567.4087
868.3576
2389.7951
2368.123
2837.8093
1133.526
              id  avgSpeed     battery timeloss routeLength  numOfStops  \
0     C6359838.0  3.600735    4.485516   109.62      673.45           2   
1   C734723628.0  5.563921   41.629497   138.52     3647.97          12   
2   C745091034.0  2.790037   10.840263   980.82     4075.61          11   
3    C74509178.0  1.558993   46.163117  2138.63     3973.37          11   
4    C74509688.0  1.574278   46.106459  1905.35     3994.44          11   
5    C74750329.0  5.090119   24.476949   260.63     3407.49           5   
6    C76194424.0  4.896042   17.009797   226.72     2727.45           7   
7   C761944419.0  5.199703   14.474859   168.66     1904.06           6   
8   C761944437.0  4.581890   17.028235   246.14     2727.45           7   
9     C764

# CSV

In [318]:
tableBattery = pd.read_csv('batteryData.csv', delimiter=';')

tableBattery.shape

(0, 11)

## settings

In [319]:
locSetting = get_last_folder(base_folder)
seedSetting = 'fgh'
trafficScaleSetting = 'asd'

#routeLengthSetting = :)
#numOfStopsSetting = :)
#tripIdSetting = :)
#avgSpeedSetting = :)
#timelossSetting = :)
#eleupSetting = 
#eledownSetting = 
#emissionSetting = :)

## Df to csv

In [320]:
for index, row in result_df.iterrows():
    row_data = {
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': row['up'],
        'elevation_down': row['down'],
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['battery']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableBattery are excluded before concatenating the DataFrames
    tableBattery = tableBattery.dropna(axis=1, how='all')

    tableBattery = pd.concat([tableBattery, temp_df], ignore_index=True)


'id': group_id,
        'avgSpeed': avg_speed,
        'batteryCapacity': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'stops': count_stops

In [321]:
tableBattery.to_csv('batteryData.csv', index=False, sep=';')
tableBattery

,loc,tripId,seed,avgSpeed,timeloss,route_length,elevation_up,elevation_down,trafficScale,numOfStops,emission
0,nap_gellert_b,C6359838.0,fgh,3.600735,109.62,673.45,8.72,-10.81,asd,2,4.485516
1,nap_gellert_b,C734723628.0,fgh,5.563921,138.52,3647.97,105.48,-68.31,asd,12,41.629497
2,nap_gellert_b,C745091034.0,fgh,2.790037,980.82,4075.61,22.54,-79.52,asd,11,10.840263
3,nap_gellert_b,C74509178.0,fgh,1.558993,2138.63,3973.37,81.46,-24.90,asd,11,46.163117
4,nap_gellert_b,C74509688.0,fgh,1.574278,1905.35,3994.44,81.32,-24.76,asd,11,46.106459
5,nap_gellert_b,C74750329.0,fgh,5.090119,260.63,3407.49,98.62,-101.92,asd,5,24.476949
6,nap_gellert_b,C76194424.0,fgh,4.896042,226.72,2727.45,20.29,-29.35,asd,7,17.009797
7,nap_gellert_b,C761944419.0,fgh,5.199703,168.66,1904.06,22.14,-16.83,asd,6,14.474859
8,nap_gellert_b,C761944437.0,fgh,4.581890,246.14,2727.45,20.25,-29.30,asd,7,17.028235
9,nap_gellert_b,C7644455.0,fgh,4.616105,113.39,1077.56,18.04,-10.59,asd,3,9.712087
